In [ ]:
import sys
import os

# set working directory to the code location
%cd ~/github/probModel

%load_ext autoreload
%autoreload
from solver import ProbPipeline
import json
config = json.loads(open("decoy.json").read())
pipeline = ProbPipeline(config)
pipeline.initialize()

In [ ]:
%autoreload

solver = pipeline.get_solver(theta=10.0, lambda_=0)
solver.s = 1e-3
solver.t = 1e2
# s * t should be less than ~0.1 (more precisely, 1.0 / (operator norm of [D; \grad]) ** 2)

In [ ]:
%autoreload
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
import IPython
ip = IPython.get_ipython()

from solver import TVSolver

from IPython import display
plt.figure(figsize=(10,15))

def loop():
    global solver
    for i in range(500):
        p, d = solver.run_single_iteration(i % 4 == 0)

        if i > 0 and i % 4 == 0:
            maximum = np.percentile(solver.w_hat[0:-1,:], 99)
            imgs = range(6) # show first 6 images
            for k, j in enumerate(imgs):
                plt.subplot(len(imgs)/3+1, 3, k+1)
                img = solver.get_image(solver.w_hat, j)
                perc = np.percentile(img, 99); img[img > perc] = perc
                fig = plt.imshow(img, vmin=0, vmax=maximum);
                plt.axis('off')

            # allows to look at animated pictures, watching intermediate results in real time
            display.clear_output(wait=True)
            display.display(plt.gcf())
        
            print i, solver.LL(solver.w_hat), p, d, solver.total_variation(solver.w_hat), solver.w_hat.sum()
            
loop()